rules of the game : https://cdn.1j1ju.com/medias/85/5e/7c-pickomino-rulebook.pdf

subject : https://team.inria.fr/polaris/files/2023/10/project_2023.pdf

#PART 1: MDP for 1 player

In [1]:
import random as rd
import itertools
import math

from dice_state import *
from mdp import *

\go faire un truc déguelasse avec dico et sets et ensuite on optimise si besoin

on considère dans toute la suite que 6 est synonyme de 'W' (worm)

a state is : a sorted list of dices rolled + a set of faces you cannot choose + the sum of dices already picked

In [3]:
#test iterator and compute_prob

it = itertools.combinations_with_replacement([1,2,3,4,5,6],3)

s = 0
for x in it:
  print(x,compute_prob(x))
  s+=compute_prob(x)

print(s)

(1, 1, 1) 0.004629629629629629
(1, 1, 2) 0.013888888888888888
(1, 1, 3) 0.013888888888888888
(1, 1, 4) 0.013888888888888888
(1, 1, 5) 0.013888888888888888
(1, 1, 6) 0.013888888888888888
(1, 2, 2) 0.013888888888888888
(1, 2, 3) 0.027777777777777776
(1, 2, 4) 0.027777777777777776
(1, 2, 5) 0.027777777777777776
(1, 2, 6) 0.027777777777777776
(1, 3, 3) 0.013888888888888888
(1, 3, 4) 0.027777777777777776
(1, 3, 5) 0.027777777777777776
(1, 3, 6) 0.027777777777777776
(1, 4, 4) 0.013888888888888888
(1, 4, 5) 0.027777777777777776
(1, 4, 6) 0.027777777777777776
(1, 5, 5) 0.013888888888888888
(1, 5, 6) 0.027777777777777776
(1, 6, 6) 0.013888888888888888
(2, 2, 2) 0.004629629629629629
(2, 2, 3) 0.013888888888888888
(2, 2, 4) 0.013888888888888888
(2, 2, 5) 0.013888888888888888
(2, 2, 6) 0.013888888888888888
(2, 3, 3) 0.013888888888888888
(2, 3, 4) 0.027777777777777776
(2, 3, 5) 0.027777777777777776
(2, 3, 6) 0.027777777777777776
(2, 4, 4) 0.013888888888888888
(2, 4, 5) 0.027777777777777776
(2, 4, 6

In [2]:
from dice_state import *
from mdp import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
# dices = (1, 2)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(state, mdp.value[state], mdp.opti[state])


(1, 3, 3, 3, 3, 4, 4, 5, 6) 0 set() 1.232827668101253 Continue with dice 4


In [4]:
c = -3 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

state = DiceState(dices, score, used)
mdp.explore(state)


print(state, mdp.value[state], mdp.opti[state])


(1, 3, 3, 3, 3, 4, 4, 5, 6) 0 set() 0.25628056121516724 4


In [5]:
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,1] + [0]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

print(len(mdp.value))
"""for i in mdp.value.keys():
    print(i, mdp.value[i])"""

0
1
2
3
4
5
0
1
2
3
4
5
0.24243906674019297
35041


'for i in mdp.value.keys():\n    print(i, mdp.value[i])'

In [6]:
c = 0 #value if you lose the turn
r = [0,0,0,0,0,0,1] + [1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0
1
2
3
4
5
0
1
2
3
4
5
0.310248006941


In [2]:
it = itertools.combinations_with_replacement([1, 1, 3, 4, 5, 6], 2)
for i in it:
    print(i)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(4, 4)
(4, 5)
(4, 6)
(5, 5)
(5, 6)
(6, 6)


In [7]:
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *


p1 = GreedyPlayer()
p2 = GreedyPlayer()

NUM_GAMES = 10000

w = 0
for _ in range(NUM_GAMES):
    if run_game(p1, p2) == PlayerTurn.PLAYER_1:
        w+=1
print(f"GreedyPlayer 1 wins with probabilty {w/NUM_GAMES} over GreedyPlayer 2")



GreedyPlayer 1 wins with probabilty 0.53 over GreedyPlayer 2
